# Amazon Bedrock AgentCore Policy - MCP 서버 정책 적용

## 개요

이 노트북은 AgentCore Runtime에 배포된 MCP 서버 타겟에 대한 **Cedar 정책 적용**을 시연합니다.

### 사전 요구사항

먼저 **`01-Setup-MCP-Runtime-Gateway.ipynb`**를 완료하여 다음을 생성하세요:
- MCP 서버가 포함된 AgentCore Runtime
- MCP 타겟이 포함된 AgentCore Gateway
- 인증을 위한 Cognito User Pool

### 학습 목표

- Policy Engine 생성 및 Gateway에 연결
- 도구 인자에 `context.input.{field}`를 사용하는 Cedar 정책 적용
- 문자열 동등 비교 (`context.input.risk_level == "low"`)
- `like` 연산자를 사용한 패턴 매칭 (`context.input.risk_level like "*low*"`)
- OR 조건 (`risk_level == "low" || risk_level == "medium"`)
- 부정 (`!(context.input.risk_level == "critical")`)

### Lambda 타겟과의 주요 차이점

| 항목 | Lambda 타겟 | MCP 서버 타겟 |
|------|-------------|---------------|
| 백엔드 | AWS Lambda 함수 | AgentCore Runtime의 MCP 서버 |
| 프로토콜 | Lambda Invoke | OAuth를 사용한 MCP over HTTP |
| 도구 탐색 | 인라인 스키마 | SynchronizeGatewayTargets API |
| 호스팅 | AWS 관리형 Lambda | AgentCore Runtime (컨테이너) |
| 인증 | N/A (내부) | OAuth2 Credential Provider |

---

## Part 1: 환경 설정

배포된 MCP Runtime에서 필요한 라이브러리와 설정을 로드합니다.

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions from common folder
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
    display_test_result,
)
from common.gateway_utils import (
    validate_and_fix_gateway_authorizer,
    attach_policy_engine_to_gateway,
    list_gateway_targets,
)
from common.policy_utils import (
    get_policy_engine,
    create_cedar_policy,
    wait_for_policy_active,
    delete_policy,
    cleanup_existing_policies,
    ensure_policy_engine,
)

print("✓ Libraries and utilities loaded")

### Step 1.1: 설정 로드

`runtime_config.json`에서 배포된 MCP Runtime 설정을, `gateway_cognito_config.json`에서 Gateway Cognito를 로드합니다.

In [ ]:
# Load Runtime Configuration
runtime_config_path = Path.cwd() / "runtime_config.json"

if not runtime_config_path.exists():
    raise FileNotFoundError(
        "runtime_config.json not found. Please run 01-Setup-MCP-Runtime-Gateway.ipynb first."
    )

with open(runtime_config_path, "r") as f:
    RUNTIME_CONFIG = json.load(f)

# Load Gateway Cognito Configuration
gateway_cognito_path = Path.cwd() / "gateway_cognito_config.json"

if not gateway_cognito_path.exists():
    raise FileNotFoundError(
        "gateway_cognito_config.json not found. Please run 01-Setup-MCP-Runtime-Gateway.ipynb first."
    )

with open(gateway_cognito_path, "r") as f:
    GATEWAY_COGNITO = json.load(f)

print("✓ Configuration files loaded")

In [ ]:
# Extract key configuration values
REGION = RUNTIME_CONFIG["region"]
GATEWAY_URL = RUNTIME_CONFIG["gateway_url"]
GATEWAY_ID = RUNTIME_CONFIG["gateway_id"]
GATEWAY_ARN = RUNTIME_CONFIG["gateway_arn"]
MCP_TARGET_ID = RUNTIME_CONFIG["target_id"]

# Gateway Cognito for API calls
CLIENT_ID = GATEWAY_COGNITO["client_id"]
CLIENT_SECRET = GATEWAY_COGNITO["client_secret"]
TOKEN_ENDPOINT = GATEWAY_COGNITO["token_endpoint"]
SCOPE = GATEWAY_COGNITO.get("scope", "")

# MCP Target Name (for tool naming)
MCP_TARGET_NAME = "RefundMCPServerTarget"

print("✓ Configuration loaded")
print(f"  REGION: {REGION}")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  GATEWAY_URL: {GATEWAY_URL}")
print(f"  GATEWAY_ARN: {GATEWAY_ARN}")
print(f"  MCP_TARGET_ID: {MCP_TARGET_ID}")
print(f"  TOKEN_ENDPOINT: {TOKEN_ENDPOINT}")

### Step 1.2: AWS 클라이언트 초기화

In [ ]:
session = boto3.Session(region_name=REGION)

sts_client = session.client("sts")
policy_client = session.client("bedrock-agentcore-control", region_name=REGION)
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

# Get Account ID
ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")
print(f"  Region: {REGION}")

### Step 1.3: Gateway 및 MCP 타겟 확인

In [ ]:
# Verify Gateway has the MCP target
targets = list_gateway_targets(gateway_control_client, GATEWAY_ID)
print(f"\n✓ Gateway targets: {len(targets)}")
for target in targets:
    print(f"  - {target.get('name', 'N/A')} (ID: {target.get('targetId')})")

---

## Part 2: Policy Engine 설정

Cedar 정책 평가를 위해 Policy Engine을 생성하고 Gateway에 연결합니다.

> **참고**: `01-Lambda-Target`에서 이미 Policy Engine이 있다면 재사용됩니다.

In [ ]:
# Check if Policy Engine already exists in runtime_config
POLICY_ENGINE_ARN = RUNTIME_CONFIG.get("policy_engine_arn")

if POLICY_ENGINE_ARN:
    POLICY_ENGINE_ID = POLICY_ENGINE_ARN.split("/")[-1]
    print(f"✓ Using existing Policy Engine: {POLICY_ENGINE_ID}")
else:
    print("Policy Engine not found. Will create one in next cell.")
    POLICY_ENGINE_ID = None

In [ ]:
# Create Policy Engine if needed
if not POLICY_ENGINE_ID:
    POLICY_ENGINE_NAME = "agentcore-policy-engine"
    
    policy_engine_info = ensure_policy_engine(
        policy_client=policy_client,
        policy_engine_name=POLICY_ENGINE_NAME
    )
    
    POLICY_ENGINE_ID = policy_engine_info["policy_engine_id"]
    POLICY_ENGINE_ARN = policy_engine_info["policy_engine_arn"]
    
    # Save to runtime_config
    RUNTIME_CONFIG["policy_engine_arn"] = POLICY_ENGINE_ARN
    with open(runtime_config_path, "w") as f:
        json.dump(RUNTIME_CONFIG, f, indent=2)
    print("✓ Policy Engine ARN saved to runtime_config.json")
else:
    # Verify existing Policy Engine
    engine = get_policy_engine(policy_client, POLICY_ENGINE_ID)
    if engine:
        print(f"✓ Policy Engine: {POLICY_ENGINE_ID}")
        print(f"  Status: {engine.get('status', 'N/A')}")
    else:
        print("✗ Policy Engine not found, creating new one...")
        POLICY_ENGINE_ID = None

### Gateway에 Policy Engine 연결

In [ ]:
# Attach Policy Engine to Gateway
attach_result = attach_policy_engine_to_gateway(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    policy_engine_arn=POLICY_ENGINE_ARN
)

if attach_result:
    print("✓ Policy Engine attached to Gateway")
else:
    print("⚠️  Failed to attach Policy Engine")

---

## Part 3: MCP 타겟 도구 이름 이해하기

MCP 서버가 Gateway 타겟으로 연결되면 도구 이름은 다음 형식을 따릅니다:

```
{TargetName}___{tool_name}
```

`refund`, `get_order`, `approve_claim` 도구가 있는 MCP 서버의 경우, Gateway 도구 이름은:

| MCP 서버 도구 | Gateway 도구 이름 |
|---------------|-------------------|
| `refund` | `RefundMCPServerTarget___refund` |
| `get_order` | `RefundMCPServerTarget___get_order` |
| `approve_claim` | `RefundMCPServerTarget___approve_claim` |

Cedar 정책에서는 다음과 같이 참조합니다:

```cedar
action == AgentCore::Action::"RefundMCPServerTarget___approve_claim"
```

In [ ]:
# Define tool names for testing
# Note: We use approve_claim for testing as it works reliably with Cedar policies
TOOL_NAME = f"{MCP_TARGET_NAME}___approve_claim"

print(f"✓ Tool name for Cedar policy: {TOOL_NAME}")

---

## Part 4: 테스트 시나리오 1 - 문자열 동등 비교

`risk_level == "low"`인 청구만 허용하는 Cedar 정책을 생성합니다.

### Cedar 정책 패턴

도구 입력 파라미터에 접근하려면 `context.input.{field}`를 사용합니다:

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level == "low"
};
```

### Step 4.0: 기존 정책 정리

In [ ]:
# Clean up existing policies for fresh start
cleanup_existing_policies(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    require_confirmation=False  # Auto delete
)

# Track created policies for cleanup
CREATED_POLICIES = []

### Step 4.1: 문자열 동등 비교 정책 생성

In [ ]:
print("=" * 70)
print("Test Scenario 1: String Equality")
print("=" * 70)

policy_name = f"string_equality_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    context.input.risk_level == "low"
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims only when risk_level == 'low'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow only low risk claims"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")
    else:
        print("⚠️  Policy did not become ACTIVE")
else:
    print("\n✗ Policy creation failed")

### Step 4.2: Bearer 토큰 획득

In [ ]:
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

print(f"✓ Token received")
print(f"  Token (first 50 chars): {token[:50]}...")

claims = decode_token(token)
print(f"\n🔍 Token Claims:")
print(f"  client_id: {claims.get('client_id', 'N/A')}")
print(f"  scope: {claims.get('scope', 'N/A')}")

### Step 4.3: 테스트 - risk_level="low" (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.1: risk_level='low' request")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-001", "amount": 500, "risk_level": "low"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='low')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "risk_level='low' should be ALLOWED")

### Step 4.4: 테스트 - risk_level="high" (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 1.2: risk_level='high' request")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-002", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "risk_level='high' should be DENIED")

---

## Part 5: 테스트 시나리오 2 - 패턴 매칭 (like)

risk_level에 "low"가 포함된 청구를 허용하는 Cedar 정책을 생성합니다.

### Cedar 정책 패턴

| 패턴 | 매칭 |
|------|------|
| `"*low*"` | "low" 포함 |
| `"low*"` | "low"로 시작 |
| `"*low"` | "low"로 끝남 |

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level like "*low*"
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 2: Pattern Matching (like)")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 5.1: 패턴 매칭 정책 생성

In [ ]:
policy_name = f"pattern_matching_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    context.input.risk_level like "*low*"
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level contains 'low'")
print("  ✓ Matches: 'low', 'very_low', 'low_priority'")
print("  ✗ No match: 'high', 'medium', 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow claims with risk_level containing 'low'"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 5.2: 테스트 - risk_level="very_low" (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.1: risk_level='very_low' (contains 'low')")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-003", "amount": 500, "risk_level": "very_low"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='very_low')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'very_low' contains 'low' - should be ALLOWED")

### Step 5.3: 테스트 - risk_level="high" (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 2.2: risk_level='high' (does not contain 'low')")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-004", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'high' does not contain 'low' - should be DENIED")

---

## Part 6: 테스트 시나리오 3 - OR 조건

risk_level이 "low" 또는 "medium"인 청구를 허용하는 Cedar 정책을 생성합니다.

### Cedar 정책 패턴

```cedar
permit(principal, action, resource)
when {
    context.input.risk_level == "low" ||
    context.input.risk_level == "medium"
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 3: OR Conditions")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 6.1: OR 조건 정책 생성

In [ ]:
policy_name = f"or_condition_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    (context.input.risk_level == "low" || context.input.risk_level == "medium")
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level is 'low' OR 'medium'")
print("  ✓ Allowed: 'low', 'medium'")
print("  ✗ Denied: 'high', 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow claims with low or medium risk"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 6.2: 테스트 - risk_level="medium" (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 3.1: risk_level='medium'")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-005", "amount": 500, "risk_level": "medium"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='medium')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'medium' matches OR condition - should be ALLOWED")

### Step 6.3: 테스트 - risk_level="high" (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 3.2: risk_level='high'")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-006", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'high' does not match OR condition - should be DENIED")

---

## Part 7: 테스트 시나리오 4 - 부정 (NOT)

risk_level이 "critical"이 아닌 청구를 허용하는 Cedar 정책을 생성합니다.

### Cedar 정책 패턴

```cedar
permit(principal, action, resource)
when {
    !(context.input.risk_level == "critical")
};
```

In [ ]:
# Clean up previous policy
print("=" * 70)
print("Test Scenario 4: Negation (NOT)")
print("=" * 70)

print("\nCleaning up previous policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)
CREATED_POLICIES.clear()

### Step 7.1: 부정 정책 생성

In [ ]:
policy_name = f"negation_policy_{int(time.time())}"

cedar_statement = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    !(context.input has risk_level && context.input.risk_level == "critical")
}};'''

print("\nCedar Policy:")
print("-" * 60)
print(cedar_statement)
print("-" * 60)
print("\nThis policy allows claims when risk_level is NOT 'critical'")
print("  ✓ Allowed: 'low', 'medium', 'high'")
print("  ✗ Denied: 'critical'")

In [ ]:
# Create the policy
policy_id = create_cedar_policy(
    policy_client=policy_client,
    policy_engine_id=POLICY_ENGINE_ID,
    policy_name=policy_name,
    cedar_statement=cedar_statement,
    description="Allow all claims except critical risk"
)

if policy_id:
    CREATED_POLICIES.append(policy_id)
    
    print("\n⏳ Waiting for policy to become ACTIVE...")
    if wait_for_policy_active(policy_client, POLICY_ENGINE_ID, policy_id):
        print("✓ Policy is ACTIVE")

### Step 7.2: 테스트 - risk_level="high" (예상: 허용)

In [ ]:
print("\n" + "=" * 70)
print("Test 4.1: risk_level='high' (not critical)")
print("=" * 70)

# Get fresh token
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE
)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-007", "amount": 500, "risk_level": "high"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='high')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("ALLOWED", outcome, "'high' is not 'critical' - should be ALLOWED")

### Step 7.3: 테스트 - risk_level="critical" (예상: 거부)

In [ ]:
print("\n" + "=" * 70)
print("Test 4.2: risk_level='critical'")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name=TOOL_NAME,
    arguments={"claim_id": "CLM-008", "amount": 500, "risk_level": "critical"}
)

print(f"\nRequest: {TOOL_NAME}(risk_level='critical')")
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
display_test_result("DENIED", outcome, "'critical' matches negation - should be DENIED")

---

## Part 8: 고급 패턴

### 복합 조건

여러 조건을 결합하여 복잡한 접근 제어 시나리오를 만들 수 있습니다.

In [ ]:
# Example: Combined conditions
print("\nAdvanced Pattern: Combined Conditions")
print("=" * 70)

combined_cedar = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has risk_level &&
    (context.input.risk_level == "low" || context.input.risk_level == "medium") &&
    !(context.input has claim_id && context.input.claim_id like "*FRAUD*")
}};'''

print("Combined Cedar Policy:")
print("-" * 60)
print(combined_cedar)
print("-" * 60)
print("\nThis policy allows when:")
print("  ✓ risk_level is 'low' OR 'medium'")
print("  AND")
print("  ✓ claim_id does NOT contain 'FRAUD'")

### 패턴 매칭 변형

| 패턴 | 매칭 | 예시 |
|------|------|------|
| `"*admin*"` | "admin" 포함 | "admin", "super_admin", "admin_user" |
| `"admin*"` | "admin"으로 시작 | "admin", "admin_user" |
| `"*admin"` | "admin"으로 끝남 | "admin", "super_admin" |
| `"team-*"` | "team-"으로 시작 | "team-alpha", "team-beta" |

In [ ]:
# Example: Wildcard pattern for claim_id
print("\nAdvanced Pattern: Wildcard for Claim ID Prefix")
print("=" * 70)

prefix_cedar = f'''permit(principal,
    action == AgentCore::Action::"{TOOL_NAME}",
    resource == AgentCore::Gateway::"{GATEWAY_ARN}")
when {{
    context.input has claim_id &&
    context.input.claim_id like "AUTO-*"
}};'''

print("Pattern Matching Cedar Policy:")
print("-" * 60)
print(prefix_cedar)
print("-" * 60)
print("\nThis policy allows:")
print("  ✓ claim_id: 'AUTO-001', 'AUTO-CLAIM-123'")
print("  ✗ claim_id: 'MANUAL-001', 'CLM-AUTO-001'")

---

## Part 9: 정리

테스트 중 생성된 정책을 삭제합니다.

In [ ]:
print("=" * 70)
print("Cleanup")
print("=" * 70)

print(f"\nDeleting {len(CREATED_POLICIES)} policies...")
for pid in CREATED_POLICIES:
    delete_policy(policy_client, POLICY_ENGINE_ID, pid)

CREATED_POLICIES.clear()
print("\n✓ Cleanup complete")

---

## 결론

이 튜토리얼에서는 MCP 서버 타겟에 대한 Cedar 정책 적용을 시연했습니다. 배운 내용:

✅ Policy Engine 생성 및 Gateway에 연결  
✅ 도구 인자에 `context.input.{field}`를 사용하는 Cedar 정책 적용  
✅ 문자열 동등 비교 (`context.input.risk_level == "low"`)  
✅ `like` 연산자를 사용한 패턴 매칭 (`context.input.risk_level like "*low*"`)  
✅ OR 조건 (`risk_level == "low" || risk_level == "medium"`)  
✅ 부정 (`!(context.input.risk_level == "critical")`)  

### 핵심 Cedar 문법 패턴

| 조건 유형 | Cedar 문법 | 예시 |
|-----------|-----------|------|
| 문자열 동등 | `context.input.field == "value"` | `risk_level == "low"` |
| 패턴 매칭 | `context.input.field like "*pattern*"` | `risk_level like "*low*"` |
| OR 조건 | `condition1 \|\| condition2` | `risk_level == "low" \|\| risk_level == "medium"` |
| 부정 | `!(condition)` | `!(risk_level == "critical")` |
| 필드 존재 확인 | `context.input has field` | `context.input has risk_level` |

### 도구 이름 형식

```
{TargetName}___{tool_name}
```

예시: `RefundMCPServerTarget___approve_claim`

### 다음 단계

1. **프로덕션 배포** - 프로덕션 MCP 서버에 이 패턴 적용
2. **JWT 클레임** - `principal.getTag()`와 결합하여 ID 기반 접근 제어 구현
3. **모니터링** - 정책 거부에 대한 CloudWatch 알람 설정